In [1]:
from datetime import datetime
import os
import time
import requests
import json

import pickle
from pathlib import Path
import traceback

In [3]:
# call API to retrieve comments of a game with a game id

# game_steamid = 1716740          # starfield
# game_name = 'starfield'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

game_steamid = 730
game_name = 'counter-strike_2'

params = {
    'json':1,
    'cursor':'*',
    'num_per_page':100,
    'language':'english',       # only english reviews
    'filter':'recent',
    'review_type':'all',
    'purchase_type':'all'

}

# api doc: https://partner.steamgames.com/doc/store/getreviews
# reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

In [4]:
# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'./{game_name}')
if game_folder.exists():
    try:
        latest_file_path = sorted(game_folder.glob('steam_reviews_*.pkl'))[-1]
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            cursor_str = reviews_reqs[-1]['cursor']
            print('Loaded:', latest_file_path)
            print('Cursor:', cursor_str)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

Loaded: counter-strike_2/steam_reviews_730_20240226235729.pkl
Cursor: AoJ4/ZfPkocDeLqVhQQ=


In [5]:
reviews_reqs[0]

{'success': 1,
 'query_summary': {'num_reviews': 100,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 1928659,
  'total_negative': 261147,
  'total_reviews': 2189806},
 'reviews': [{'recommendationid': '159325319',
   'author': {'steamid': '76561199098431188',
    'num_games_owned': 4,
    'num_reviews': 1,
    'playtime_forever': 1555,
    'playtime_last_two_weeks': 831,
    'playtime_at_review': 1550,
    'last_played': 1708956238},
   'language': 'english',
   'review': 'causes sleepless nights 10/10',
   'timestamp_created': 1708955969,
   'timestamp_updated': 1708955969,
   'voted_up': True,
   'votes_up': 0,
   'votes_funny': 0,
   'weighted_vote_score': 0,
   'comment_count': 0,
   'steam_purchase': False,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '159325241',
   'author': {'steamid': '76561199161404273',
    'num_games_owned': 0

In [6]:
len(reviews_reqs)

2550

In [7]:
review_list = []

datetime_until = datetime(2024, 1, 1, 0, 0, 0)

for reviews_req in reviews_reqs:
    review_list.extend(reviews_req['reviews'])

# filter by unique 'recommendationid'
unique_review_list = []
unique_review_ids = set()

for review in review_list:
    if review['recommendationid'] in unique_review_ids:
        continue

    if review['timestamp_created'] < datetime_until.timestamp():
        continue

    unique_review_list.append(review)
    unique_review_ids.add(review['recommendationid'])

len(unique_review_list)

33977

Very similar to the time I read the number of reviews: 96741, at 20240211:000030

In [8]:
# save the unique reviews

with open(f'./{game_name}/steam_reviews_{game_steamid}_unique.pkl', 'wb') as f:
    pickle.dump(unique_review_list, f)

The number excludes off-topic reviews (??)

---

In [24]:
reviews_req.json()

{'success': 1,
 'query_summary': {'num_reviews': 100},
 'reviews': [{'recommendationid': '153833346',
   'author': {'steamid': '76561198170136267',
    'num_games_owned': 343,
    'num_reviews': 24,
    'playtime_forever': 2595,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 2595,
    'last_played': 1694489855},
   'language': 'english',
   'review': 'This is the worst parts of bethesda games without the best parts of bethesda games',
   'timestamp_created': 1702867677,
   'timestamp_updated': 1702867677,
   'voted_up': False,
   'votes_up': 7,
   'votes_funny': 0,
   'weighted_vote_score': '0.540235579013824463',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '153833339',
   'author': {'steamid': '76561197991612208',
    'num_games_owned': 2661,
    'num_reviews': 40,
    'playtime_forever': 852,
    'playtime_

In [6]:
with open('reviews.json', 'w') as f:
    json.dump(reviews_req.json(), f, indent=2)

In [4]:
print(reviews_req.json()['reviews'][2]['review'])

Was initially disappointed with this game but its come along way and I seriously enjoy it. Cant wait for CK to be added since modding it right now is a massive pain
